###### ### The University of Melbourne, School of Computing and Information Systems
# COMP30027 Machine Learning, 2023 Semester 1

## Assignment 1: Music genre classification with naive Bayes


**Student ID(s):**     `1174154`


This iPython notebook is a template which you will use for your Assignment 1 submission.

Marking will be applied on the four functions that are defined in this notebook, and to your responses to the questions at the end of this notebook (Submitted in a separate PDF file).

**NOTE: YOU SHOULD ADD YOUR RESULTS, DIAGRAMS AND IMAGES FROM YOUR OBSERVATIONS IN THIS FILE TO YOUR REPORT (the PDF file).**

You may change the prototypes of these functions, and you may write other functions, according to your requirements. We would appreciate it if the required functions were prominent/easy to find.

**Adding proper comments to your code is MANDATORY. **

In [16]:
import pandas as pd
import numpy as np
import scipy.stats as ss
import math

EPSILON = 0.00000001
BASE = -99999999

In [3]:
def calc_likelihood(data):
    # Create an empty dictionary to store the likelihoods
    likelihood = {}
    unique_labels = np.unique(data.values[:, -1])
    features_list = data.columns[1:]
    
    for feature in features_list:
        
        
        feature_values = np.unique(data[feature])
        
        # Create a nested dictionary for each feature
        likelihood[feature] = {}

        for label in unique_labels:
            # Create a nested dictionary for each label
            likelihood[feature][label] = {}

            for value in feature_values:
                count = len(data[(data['label'] == label) & (data[feature] == value)])
                total_count = len(data[(data['label'] == label)])

                likelihood[feature][label][value] = count / total_count
                print(f"Likelihood of {feature}={value} given label {label} is {likelihood[feature][label][value]:.4f}")
                
    return likelihood

In [4]:
# This function should prepare the data by reading it from a file and converting it into a useful format for training and testing
def preprocess():
    # test data classifying
    test_data_df = pd.read_csv('COMP30027_2023_asst1_data\gztan_test.csv')
    
    # train data classifying
    train_data_df = pd.read_csv('COMP30027_2023_asst1_data\gztan_train.csv')

    X_train = train_data_df.iloc[:, 1:-1]
    y_train = train_data_df.iloc[:,-1]
    X_test = test_data_df.iloc[:, 1:-1]
    y_test = test_data_df.iloc[:,-1]
    
    return X_train, y_train, X_test, y_test, test_data_df, train_data_df
    # return X_train, y_train, train_data_df

In [5]:
# This function should calculate prior probabilities and likelihoods from the training data and using
# them to build a naive Bayes model

def train():
    X_train, y_train, X_test, y_test, test_data_df, train_data_df = preprocess()
    # X_train, y_train, train_data_df= preprocess()

    # calculate prior probability by getting unique labels and 
    prior_prob = {}

    unique_labels, counts = np.unique(y_train, return_counts=True)
    n = counts.sum()

    for i in range(len(unique_labels)):
        prior_prob[unique_labels[i]] = (counts[i] / n).round(3)

    # calculate miximum likelihood
    # gives attributes for the trainning dataset
    attribute_lists = X_train.columns
    likelihood_parameters = list()

    # for each label, correspond with a attribute, calculate the mean and sd for this combination, which will be used to calculate likelihoods P(attribute x1 | class ci)
    for label in unique_labels:
        data_with_label = train_data_df[train_data_df['label'] == label]

        for attribute in attribute_lists:
            train_attribute_mean = data_with_label[attribute].mean()
            train_attribute_std = data_with_label[attribute].std()
            likelihood_parameters.append([train_attribute_mean, train_attribute_std])

        # for value in X_train[attribute]:
        #     prior = ss.norm(train_attribute_mean, train_attribute_std).cdf(value)
    # print(likelihood_parameters)
    return prior_prob, likelihood_parameters, X_test, y_test, test_data_df
    
train()

({'blues': 0.101,
  'classical': 0.1,
  'country': 0.105,
  'disco': 0.098,
  'hiphop': 0.099,
  'jazz': 0.102,
  'metal': 0.1,
  'pop': 0.096,
  'reggae': 0.108,
  'rock': 0.091},
 [[0.33942086865872517, 0.0609267945615699],
  [0.09071345930850064, 0.006825797861205192],
  [0.13425552426480955, 0.05074081991525121],
  [0.002942114768773962, 0.0017380904185093503],
  [1735.776363600462, 524.0626832519889],
  [291671.42012162256, 184436.74844814203],
  [1924.312221817604, 335.8715240464701],
  [111107.17539984563, 79082.20950613037],
  [3641.0129312081076, 1171.2174689863093],
  [1476825.9841657332, 1220623.644004171],
  [0.0799962669210039, 0.03553295479719767],
  [0.0016931551428339775, 0.0009848696371030491],
  [-0.00040053633515676974, 0.0014906732719559676],
  [0.013149942716951517, 0.009005044622685547],
  [-0.0005407970406990162, 0.0014308344172170508],
  [0.004522978351523275, 0.004353666711584151],
  [120.72394578228547, 31.33051081619725],
  [-166.2502325081531, 84.53167761001

In [20]:
# This function should predict classes for new items in a test dataset

def predict():
    prior_prob, likelihood_parameters, X_test, y_test, test_data_df = train()

    # get the labels and attributes for our test dataset
    unique_labels = list(prior_prob.keys())
    attribute_lists = X_test.columns

    choice_list = list()
    for index, row in X_test.iterrows():
        row_len = len(row)
        
        highest_posterior = BASE
        best_choice = None
        for i in range(len(unique_labels)):
            # take a label and find the prior probability for this label
            label = unique_labels[i]
            prior = prior_prob[label]

            # parameters for each attribute under this label
            correspoding_parameters = likelihood_parameters[int(i*row_len) : int((i+1)*row_len)]

            posterior = math.log(prior)
            for j in range(len(attribute_lists)):
                # attribute value and parameters for this attribute
                attribute = attribute_lists[j]
                parameters = correspoding_parameters[j]
                
                # find likelihood using values invoved
                value = row[attribute]
                max_likelihood = ss.norm(parameters[0], parameters[1]).cdf(value)
                
                # if the max_likelihood is very close to zero, we can use epsilon to take log
                if (max_likelihood <= EPSILON):
                    posterior += math.log(EPSILON)
                else:
                    posterior += math.log(max_likelihood)
            
            if (posterior > highest_posterior):
                highest_posterior = posterior
                best_choice = label
        
        choice_list.append(best_choice)
    return choice_list, y_test
# predict()

In [24]:
# This function should evaliate the prediction performance by comparing your model’s class outputs to ground
# truth labels

def evaluate():
    choice_list, y_test = predict()

    n = len(y_test)
    correct = 0

    for i in range(len(choice_list)):
        correct += (choice_list[i] == y_test[i])
    
    return (correct / n)

evaluate()

0.305

## Task 1. Pop vs. classical music classification

#### NOTE: you may develope codes or functions to help respond to the question here, but your formal answer must be submitted separately as a PDF.

### Q1
Compute and report the accuracy, precision, and recall of your model (treat "classical" as the "positive" class).

### Q2
For each of the features X below, plot the probability density functions P(X|Class = pop) and P(X|Class = classical). If you had to classify pop vs. classical music using just one of these three features, which feature would you use and why? Refer to your plots to support your answer.
- spectral centroid mean
- harmony mean
- tempo

## Task 2. 10-way music genre classification

#### NOTE: you may develope codes or functions to help respond to the question here, but your formal answer must be submitted separately as a PDF.

### Q3
Compare the performance of the full model to a 0R baseline and a one-attribute baseline. The one-attribute baseline should be the best possible naive Bayes model which uses only a prior and a single attribute. In your write-up, explain how you implemented the 0R and one-attribute baselines.

### Q4
Train and test your model with a range of training set sizes by setting up your own train/test splits. With each split, use cross-fold validation so you can report the performance on the entire dataset (1000 items). You may use built-in functions to set up cross-validation splits. In your write-up, evaluate how model performance changes with training set size.

### Q5
Implement a kernel density estimate (KDE) naive Bayes model and compare its performance to your Gaussian naive Bayes model. You may use built-in functions and automatic ("rule of thumb") bandwidth selectors to compute the KDE probabilities, but you should implement the naive Bayes logic yourself. You should give the parameters of the KDE implementation (namely, what bandwidth(s) you used and how they were chosen) in your write-up.

### Q6
Modify your naive Bayes model to handle missing attributes in the test data. Recall from lecture that you can handle missing attributes at test by skipping the missing attributes and computing the posterior probability from the non-missing attributes. Randomly delete some attributes from the provided test set to test how robust your model is to missing data. In your write-up, evaluate how your model's performance changes as the amount of missing data increases.